# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *

# Load files and edit datasets

LAM output analysis.

Simu irr et simu no-irr.

In [ ]:
# lon_min=-10
# lon_max=4
# lat_min=34
# lat_max=46

In [ ]:
hf_dir='../../../JZ_simu_outputs/LAM/HF_outputs'

## LMDZ

In [ ]:
#noirr
filename = '{}/noirr/sim265/*_HF_histhf.nc'.format(hf_dir)

lmdz_noirr0 = xr.open_mfdataset(filename)
lmdz_noirr = lmdz_noirr0.rename({'time_counter':'time'})
lmdz_noirr = lmdz_noirr.assign_coords(time_decimal=lmdz_noirr.time.dt.hour + lmdz_noirr.time.dt.minute / 60)
lmdz_noirr.attrs['name'] = 'no_irr'
lmdz_noirr.attrs['plot_color']='red'
lmdz_noirr = lmdz_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_noirr['sens']=-lmdz_noirr['sens']
lmdz_noirr['flat']=-lmdz_noirr['flat']

lmdz_noirr['ground_level'] = lmdz_noirr['phis'] / 9.81
lmdz_noirr['ground_level'].attrs['units'] = 'm'

lmdz_noirr=add_wind_speed(lmdz_noirr)
lmdz_noirr=add_wind_direction(lmdz_noirr)
lmdz_noirr=add_wind_10m(lmdz_noirr)

#make ovap unit g/kg
lmdz_noirr['ovap'] = lmdz_noirr['ovap']*1000
lmdz_noirr['ovap'].attrs['units'] = 'g/kg'

lmdz_noirr

In [ ]:
#irr
filename = '{}/irr/sim266/*_HF_histhf.nc'.format(hf_dir)

lmdz_irr0 = xr.open_mfdataset(filename)
lmdz_irr =lmdz_irr0.rename({'time_counter':'time'})
lmdz_irr = lmdz_irr.assign_coords(time_decimal=lmdz_irr.time.dt.hour + lmdz_irr.time.dt.minute / 60)
lmdz_irr.attrs['name'] = 'irr'
lmdz_irr.attrs['plot_color']='#0C5DA5'
lmdz_irr = lmdz_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_irr['sens']=-lmdz_irr['sens']
lmdz_irr['flat']=-lmdz_irr['flat']

lmdz_irr['ground_level'] = lmdz_irr['phis'] / 9.81
lmdz_irr['ground_level'].attrs['units'] = 'm'

lmdz_irr=add_wind_speed(lmdz_irr)
lmdz_irr=add_wind_direction(lmdz_irr)
lmdz_irr=add_wind_10m(lmdz_irr)

#make ovap unit g/kg
lmdz_irr['ovap'] = lmdz_irr['ovap']*1000
lmdz_irr['ovap'].attrs['units'] = 'g/kg'

lmdz_irr

## ORCHIDEE

In [ ]:
filename = '{}/noirr/*HF_sechiba_out_2.nc'.format(hf_dir)

orc_noirr0 = xr.open_mfdataset(filename)
orc_noirr = orc_noirr0.rename({'time_counter':'time'})
orc_noirr = orc_noirr.assign_coords(time_decimal=orc_noirr.time.dt.hour + orc_noirr.time.dt.minute / 60)
orc_noirr.attrs['name'] = 'no_irr'
orc_noirr.attrs['plot_color']='red'
orc_noirr = orc_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_noirr['evapot']=orc_noirr['evapot'] * 3600 * 24
orc_noirr['evapot'].attrs['units'] = 'mm/d'

orc_noirr

In [ ]:
filename = '{}/irr/*HF_sechiba_out_2.nc'.format(hf_dir)

orc_irr0 = xr.open_mfdataset(filename)
orc_irr = orc_irr0.rename({'time_counter':'time'})
orc_irr = orc_irr.assign_coords(time_decimal=orc_irr.time.dt.hour + orc_irr.time.dt.minute / 60)
orc_irr.attrs['name'] = 'irr'
orc_irr.attrs['plot_color']='#0C5DA5'
orc_irr = orc_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_irr['evapot']=orc_irr['evapot'] * 3600 * 24
orc_irr['evapot'].attrs['units'] = 'mm/d'

orc_irr

## Obs
### RS

In [ ]:
filename = '../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_RS_L2/LIAISE_LA-CENDROSA_CNRM_RS-ascent_L2_20210722-1200_V2.nc'
# LIAISE_LA-CENDROSA_CNRM_RS-ascent_L2_20210721*.nc'
Cendrosa_RS = xr.open_mfdataset(filename)
Cendrosa_RS.attrs['name'] = 'Cendrosa_RS'
Cendrosa_RS.attrs['plot_color']='black'

# Rename vars
rename_dict = {'windSpeed':'wind_speed', 
               'windDirection':'wind_direction',
               }
Cendrosa_RS = Cendrosa_RS.rename(rename_dict)

# Calculate potential temperature
Cendrosa_RS['theta'] = Cendrosa_RS['temperature'] * (1000 / Cendrosa_RS['pressure']) ** 0.286
Cendrosa_RS['theta'].attrs['units'] = 'K'

# Calculate specific humidity
Cendrosa_RS['ovap'] = 1000* Cendrosa_RS['mixingRatio'] / (1000 + Cendrosa_RS['mixingRatio'])
Cendrosa_RS['ovap'].attrs['units'] = 'g/kg'

Cendrosa_RS

In [ ]:
# Read the data from the text file
file_path = '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_radiosondes_L1/LIAISE_ELS-PLANS_UKMO_RADIOSONDEPROFILE_L1-20210722-120040_V1.0.txt'
ElsPlans_RS = txtRS_to_xarray(file_path)
ElsPlans_RS.attrs['name'] = 'ElsPlans_RS'
ElsPlans_RS.attrs['plot_color']='black'
rename_dict = {
    'Time':'time',
    'Height':'altitude',
    'Pres':'pressure',
    'Temp':'temperature',
    'RH':'humidity',
    'Dewp':'dewPoint',
    'MixR':'mixingRatio',
    'Dir':'wind_direction',
    'Speed':'wind_speed',
}
ElsPlans_RS = ElsPlans_RS.rename(rename_dict)

# Calculate potential temperature
ElsPlans_RS['temperature'] = ElsPlans_RS['temperature'] + 273.15
ElsPlans_RS['temperature'].attrs['units'] = 'K'

ElsPlans_RS['theta'] = ElsPlans_RS['temperature'] * (1000 / ElsPlans_RS['pressure']) ** 0.286
ElsPlans_RS['theta'].attrs['units'] = 'K'

# Calculate specific humidity
ElsPlans_RS['ovap'] = 1000* ElsPlans_RS['mixingRatio'] / (1000 + ElsPlans_RS['mixingRatio'])
ElsPlans_RS['ovap'].attrs['units'] = 'g/kg'

ElsPlans_RS

## Masking

### IP continental subdomain

In [ ]:
# Define masks
 
con_mask_2D_ORC= orc_noirr.mean(dim='time')['Contfrac']>0.95
con_mask_2D_ORC.attrs['name']='Continental mask (95%)'

ip_polygon_mask_2D=polygon_to_mask(orc_noirr.mean(dim="time"), iberian_peninsula)
ip_polygon_mask_2D.attrs['name']='Iberian Peninsula polygon mask'

#combine two masks
ip_mask = con_mask_2D_ORC * ip_polygon_mask_2D
ip_mask.attrs['name']='IPmask'

In [ ]:
#ORC
ip_orc_noirr = apply_2Dmask_to_dataset(orc_noirr, ip_mask, dsname='no_irr')
ip_orc_irr = apply_2Dmask_to_dataset(orc_irr, ip_mask, dsname='irr')

In [ ]:
#LMDZ
ip_lmdz_noirr = apply_2Dmask_to_dataset(lmdz_noirr, ip_mask, dsname='no_irr')
ip_lmdz_irr = apply_2Dmask_to_dataset(lmdz_irr, ip_mask, dsname='irr')

### high_irr vs low_irr

In [ ]:
#one grid cell
#highirr
Cendrosa_lon=0.6
Cendrosa_lat=41.7
#lowirr
ElsPlans_lon=1.1
ElsPlans_lat=41.5

In [ ]:
cen_lmdz_noirr = lmdz_noirr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_lmdz_noirr.attrs['name']='La Cendrosa no_irr'
cen_lmdz_irr = lmdz_irr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_lmdz_irr.attrs['name']='La Cendrosa irr'
cen_orc_noirr = orc_noirr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_orc_noirr.attrs['name']='La Cendrosa no_irr'
cen_orc_irr = orc_irr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_orc_irr.attrs['name']='La Cendrosa irr'

In [ ]:
els_lmdz_noirr = lmdz_noirr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_lmdz_noirr.attrs['name']='Els Plans no_irr'
els_lmdz_irr = lmdz_irr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_lmdz_irr.attrs['name']='Els Plans irr'
els_orc_noirr = orc_noirr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_orc_noirr.attrs['name']='Els Plans no_irr'
els_orc_irr = orc_irr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_orc_irr.attrs['name']='Els Plans irr'

In [ ]:
#add attr linestyle -- to els_ datasets
els_lmdz_noirr.attrs['linestyle']='--'
els_lmdz_irr.attrs['linestyle']='--'
els_orc_noirr.attrs['linestyle']='--'
els_orc_irr.attrs['linestyle']='--'
#add attr linestyle - to cen_ datasets
cen_lmdz_noirr.attrs['linestyle']='-'
cen_lmdz_irr.attrs['linestyle']='-'
cen_orc_noirr.attrs['linestyle']='-'
cen_orc_irr.attrs['linestyle']='-'

In [ ]:
cendrosa_gl= cen_lmdz_noirr['ground_level'].mean(dim='time').values.item()
elsplans_gl= els_lmdz_noirr['ground_level'].mean(dim='time').values.item()
cendrosa_gl,  elsplans_gl

### Golden days

In [ ]:
start_time = '2021-07-21T04:30:00.000000000'
end_time='2021-07-22T22:30:00.000000000'

In [ ]:
gold_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, start_time, end_time)
gold_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, start_time, end_time)
gold_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, start_time, end_time)
gold_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, start_time, end_time)

gold_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, start_time, end_time)
gold_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, start_time, end_time)
gold_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, start_time, end_time)
gold_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, start_time, end_time)

# Maps

In [ ]:
var='sens'
ds=ip_lmdz_irr

vmin=0
vmax=1000
# vmin,vmax = None, None
cmap=myterrain
# title='off'
title=None
# title='Evaporative fraction (irr, JJA)'
# clabel='Irrigation (mm/d)'
clabel=None
map_ave(ds, var, vmin=vmin, vmax=vmax, cmap=cmap,
        title=title, clabel=clabel)

In [ ]:
#Diff map
var='t2m'
ds1=ip_lmdz_irr
ds2=ip_lmdz_noirr

vmax= 0.5
vmin=-0.5
# vmin,vmax = None, None
cmap=emb_neutral
title=None
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, 
             sig=False,
             title=title, clabel=clabel)

In [ ]:
#Relative diff map
var='sens'
ds1=ip_lmdz_irr
ds2=ip_lmdz_noirr
maxvalue= 30
minvalue=-30
# minvalue, maxvalue = None, None

cmap=emb_neutral

map_rel_diff_ave(ds1, ds2, var, cmap=cmap, vmin=minvalue, vmax=maxvalue)

In [ ]:
#Display map of var for ds on restricted area
var='ground_level'
ds = ip_orc_irr
ds = lmdz_irr

latmin=41.35
latmax=41.9
lonmin=0.6
lonmax=1.4
latmin=40.5
latmax=43.5
lonmin=-3
lonmax=3
cmap=myterrain
vmin=0
vmax=1000
# vmin, vmax=None, None
# title='Irrigation (July 2021, mm/d)'
title=None

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')

map_plotvar(plotvar, cmap=cmap, vmin=vmin,vmax=vmax, title=title)

plt.plot(Cendrosa_lon, Cendrosa_lat, 'ro', markersize=10)
plt.plot(ElsPlans_lon, ElsPlans_lat, 'ro', markersize=10)

# Time series and diurnal cycle

In [ ]:
var='irrigmap_dyn'

ds1=gold_cen_lmdz_irr
ds2=gold_cen_lmdz_noirr
ds3=gold_els_lmdz_irr
ds4=gold_els_lmdz_noirr

ds1=gold_cen_orc_irr
ds2=gold_cen_orc_noirr
ds3=gold_els_orc_irr
ds4=gold_els_orc_noirr

ds_list=[ds1, ds2, ds3, ds4]
# ds_list=[ds1]

vmin=0
vmax=0.17
vmin, vmax=None, None
title='{} ({})'.format(var, ds1[var].attrs['units'])

title=None
time_series_ave(ds_list, var, ds_colors=True, ds_linestyle=True, title=title, vmin=vmin, vmax=vmax)

In [ ]:
#en un point precis
lon=Cendrosa_lon
lat=Cendrosa_lat

lon=ElsPlans_lon
lat=ElsPlans_lat
# lon=-2
# lat=41
var='s_pblh'
ds1=lmdz_noirr
ds2=lmdz_irr
# ds1=orc_noirr
# ds2=orc_irr
ds_list=[ds1, ds2]
time_series_lonlat(ds_list, var, lon, lat)

In [ ]:
var='s_pblh'

ds1=cen_lmdz_noirr
ds2=cen_lmdz_irr
# ds1=ip_orc_noirr
# ds2=ip_orc_irr
ds_list=[ds1, ds2]#, ds3]
# ds_list=[ds2]

title=None
diurnal_cycle_ave(ds_list, var, title='{} ({})'.format(var, ds1[var].attrs['units']))

In [ ]:
var='sens'

ds1=ip_lmdz_noirr
ds2=ip_lmdz_irr
# ds1=ip_orc_noirr
# ds2=ip_orc_irr

ds_list=[ds1, ds2]#, ds3]
# ds_list=[ds2]

title=None

lon=Cendrosa_lon
lat=Cendrosa_lat

# diurnal_cycle_lonlat(ds_list, var, lon=lon, lat=lat)

lon=ElsPlans_lon
lat=ElsPlans_lat

# diurnal_cycle_lonlat(ds_list, var, lon=lon, lat=lat)

# Energy budget

In [ ]:
## FOR ONE SITE, DISTINGUISHING BY SIM
ds_lmdz_irr = cen_lmdz_irr
ds_orc_irr  = cen_orc_irr
ds_lmdz_noirr = cen_lmdz_noirr
ds_orc_noirr  = cen_orc_noirr
title='La Cendrosa'
energy_budget_dc(ds_lmdz_irr, ds_lmdz_noirr, ds_orc_irr, ds_orc_noirr, title=title, lab1='irr', lab2='noirr')

ds_lmdz_irr = els_lmdz_irr
ds_orc_irr  = els_orc_irr
ds_lmdz_noirr = els_lmdz_noirr
ds_orc_noirr  = els_orc_noirr
title='Els Plans'
energy_budget_dc(ds_lmdz_irr, ds_lmdz_noirr, ds_orc_irr, ds_orc_noirr, title=title, lab1='irr', lab2='noirr')

In [ ]:
## FOR ONE SIM, DISTINGUISHING BY SITE
ds_lmdz1 = cen_lmdz_irr
ds_orc1 = cen_orc_irr
ds_lmdz2 = els_lmdz_irr
ds_orc2 = els_orc_irr
title = 'Irrigated sim'
energy_budget_dc(ds_lmdz1, ds_lmdz2, ds_orc1, ds_orc2, title=title, lab1='La Cendrosa', lab2='Els Plans')

ds_lmdz1 = cen_lmdz_noirr
ds_orc1 = cen_orc_noirr
ds_lmdz2 = els_lmdz_noirr
ds_orc2 = els_orc_noirr
title = 'Non-irrigated sim'
energy_budget_dc(ds_lmdz1, ds_lmdz2, ds_orc1, ds_orc2, title=title, lab1='La Cendrosa', lab2='Els Plans')

# Vertical profiles

In [ ]:
## Sims only

In [ ]:
var='theta'
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

# day='2021-07-22'
# ds11=filter_xarray_by_day(ds1, day)
# ds21=filter_xarray_by_day(ds2, day)
# ds31=filter_xarray_by_day(ds3, day)
# ds41=filter_xarray_by_day(ds4, day)

# ds11=mean_dataset(ds1)
# ds21=mean_dataset(ds2)
# ds31=mean_dataset(ds3)
# ds41=mean_dataset(ds4)

# ds_list=[ds11, ds21, ds31, ds41]

# ds_list=[ds1, ds2, ds3, ds4]
ds_list=[ds3, ds4]
# ds_list=[ds1]

obs_ds_list=[Cendrosa_RS]
obs_ds_list=None

altmax=3000

title=None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_local_mean(ds_list, var, title=title, altmax=altmax,
                            obs_ds_list=obs_ds_list,
                            nbins=None,
                            substract_gl=True,
                            )

In [ ]:
def profile_altitude_multipletimes(ds_list, var, times, altmin=0, altmax=2000):
    n_ax = len(times)
    fig, axs = plt.subplots(1, n_ax, figsize=(5*n_ax, 6))
    # Flatten axs only if it's an array (i.e., more than one subplot)
    axes = axs.flatten() if isinstance(axs, np.ndarray) else [axs]
    for i, time in enumerate(times):
        hour=times_correspondance[time]
        title = f"{var} at {hour}"
        
        # Filter datasets by the specified time and plot
        ds_list_tmp = [ds.where(ds['time_decimal']==time) for ds in ds_list]
        profile_altitude_local_mean(ds_list_tmp,
                                    var,
                                    ax=axes[i],
                                    title=title,
                                    altmin=altmin,
                                    altmax=altmax
        )

In [ ]:
var='theta'
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

ds_list=[ds1, ds2, ds3, ds4]
# ds_list=[ds3,ds4]
# ds_list=[ds1]

altmin=-0
altmax=2500

times=[1.5, 4.5, 7.5, 10.5, 13.5, 16.5, 19.5, 22.5]
times=[4.5, 7.5, 10.5, 13.5, 16.5, 19.5]
# times=[13.5]

xmin=-0.
xmax=9
# xmin, xmax=None, None

profile_altitude_multipletimes_mean(ds_list, var, times, altmin=altmin, altmax=altmax,
                                    xmin=xmin, xmax=xmax,
                                    substract_gl=True)

## With obs

In [ ]:
#obs alone
var='theta'
ds1=Cendrosa_RS
ds2=ElsPlans_RS
ds_list=[ds2]

altmax=2500

title=ds1.attrs['name']

profile_altitude_obs(ds_list, var, title=title, altmax=altmax,
                       nbins=None,
                       substract_gl=True,
                       )

In [ ]:
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

timestamp='2021-07-22T13:00:00.000000000'

ds_list=[ds1, ds2]#, ds3, ds4]
# ds_list=[ds3, ds4]
# ds_list=[ds3]

obs_ds_list=[Cendrosa_RS]
obs_ds_list=[ElsPlans_RS]
# obs_ds_list=None

altmax=2500
title=None

var='wind_direction'
var, xmin, xmax = 'theta', 306, 314
var, xmin, xmax = 'ovap', 2, 14
var, xmin, xmax = 'wind_speed', 0, 9
var, xmin, xmax = 'wind_direction', 0, 360
# xmin, xmax = None, None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_local_timestamp(ds_list, var, timestamp, title=title, altmax=altmax, xmin=xmin, xmax=xmax, 
                            obs_ds_list=obs_ds_list,
                            nbins=5,
                            substract_gl=True,
                            )